In [1]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install xlrd

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [200]:
import openpyxl
import pandas as pd
xlsxFile = '2020년 식수집계.xlsx'
sheetList = []
wb = openpyxl.load_workbook(xlsxFile)
for i in range(len(wb.sheetnames)-6):
    if i%2==0:
        sheetList.append(wb.sheetnames[i])
        
ppl=[]
for i in range(len(sheetList)):
    ppl.append(pd.read_excel('2020년 식수집계.xlsx',index_col=0,sheet_name=sheetList[i]).iloc[1:33,:1])
for i in range(len(ppl)):
    ppl[i].columns=ppl[i].iloc[0,:]
    ppl[i]=ppl[i].iloc[1:,:]    
for i in range(1,len(ppl)):
    ppl[0]=pd.concat([ppl[0],ppl[i]])
ppl[0]=ppl[0].reset_index()
ppl[0]=ppl[0].rename(columns={'index':'timestamp','중식':'target_value'})
data=ppl[0]
data['timestamp']=data['timestamp'].apply(lambda x:str(x))
data=data.dropna()
data=data.reset_index().drop('index',axis=1)
data['timestamp']=data['timestamp'].apply(lambda x:pd.to_datetime(x))
data #2020-01-01-2020-11-30

날 짜,timestamp,target_value
0,2020-01-01,30
1,2020-01-02,480
2,2020-01-03,474
3,2020-01-04,110
4,2020-01-05,15
...,...,...
330,2020-11-26,0
331,2020-11-27,0
332,2020-11-28,0
333,2020-11-29,0


In [201]:
import openpyxl
import pandas as pd
xlsxFile = '2019년 식수집계.xlsx'
sheetList = []
wb = openpyxl.load_workbook(xlsxFile)
for i in range(len(wb.sheetnames)-2):
    sheetList.append(wb.sheetnames[i])
ppl=[]
for i in range(0,7):
    ppl.append(pd.read_excel('2019년 식수집계.xlsx',index_col=0,sheet_name=sheetList[i]).iloc[1:34,:1])
for i in range(11,15):
    ppl.append(pd.read_excel('2019년 식수집계.xlsx',index_col=0,sheet_name=sheetList[i]).iloc[1:34,:1])
ppl.append(pd.read_excel('2019년 식수집계.xlsx',index_col=0,sheet_name=sheetList[7]).iloc[1:17,:1])  
ppl.append(pd.read_excel('2019년 식수집계.xlsx',index_col=0,sheet_name=sheetList[9]).iloc[1:13,:1])  
ppl.append(pd.read_excel('2019년 식수집계.xlsx',index_col=0,sheet_name='2019.8미가').iloc[16:34,:1])
ppl.append(pd.read_excel('2019년 식수집계.xlsx',index_col=0,sheet_name=sheetList[10]).iloc[16:32,:1])
for i in range(len(ppl)-2):
    ppl[i].columns=ppl[i].iloc[0,:]
    ppl[i]=ppl[i].iloc[1:,:]
for i in range(1,len(ppl)-2):
    ppl[0]=pd.concat([ppl[0],ppl[i]])
ppl[0]=ppl[0].reset_index() 
ppl[0]=ppl[0].rename(columns={'index':'timestamp'})  
for i in range(13,15):
    ppl[i]=ppl[i].reset_index()  
    ppl[i].columns=['날 짜','중식']
    ppl[i]=ppl[i].iloc[1:,:]  
ppl[13]=ppl[13][['날 짜','중식']]
for i in range(13,15):
    ppl[0]=pd.concat([ppl[0],ppl[i]])   

data2=ppl[0]
data2=data2[['timestamp','중식']]
data2=data2.dropna()
data2=data2.reset_index().drop('index',axis=1) #2019-01-01 - 2019-12-31
data2=data2[data2['timestamp']!='합계']
data2['timestamp']=data2['timestamp'].apply(lambda x:pd.to_datetime(x))
ppl[13]=ppl[13].rename(columns={'날 짜':'timestamp'}) 
data2=pd.concat([data2,ppl[13]])
data2=data2.sort_values('timestamp')
data2=data2.reset_index().drop('index',axis=1)
data2=data2.drop([316,319])
ppl[14]=ppl[14].rename(columns={'날 짜':'timestamp'}) 
data2=pd.concat([data2,ppl[14]])
data2=data2.sort_values('timestamp')
data2=data2.reset_index().drop('index',axis=1)
data2.loc[363,'timestamp']='2019-09-13'
data2.loc[364,'timestamp']='2019-09-14'
data2.loc[365,'timestamp']='2019-09-12'
data2.loc[366,'timestamp']='2019-09-15'
data2.loc[366,'중식']=0
data2.loc[363,'중식']=0
data2.loc[364,'중식']=0
data2.loc[365,'중식']=0
data2['timestamp']=data2['timestamp'].apply(lambda x:pd.to_datetime(x))
data2=data2.sort_values('timestamp')
data2=data2.reset_index().drop('index',axis=1)
data2=data2.drop(227)
data2=data2.rename(columns={'중식':'target_value'}) 

In [202]:
data2[data2['timestamp']=='2019-08-16']

,timestamp,target_value
228,2019-08-16,450


In [203]:
data=pd.concat([data,data2]) 
data=data.sort_values('timestamp')

In [204]:
data=data.reset_index().drop('index',axis=1)#2019-01-01 - 2020-10-30
data=data.dropna()
data['timestamp']=data['timestamp'].apply(lambda x:pd.to_datetime(x))

In [205]:
data['timestamp']=data['timestamp'].apply(lambda x:pd.to_datetime(x))
data['target_value']=data['target_value'].apply(lambda x: int(x))
data['dayofweek_mean']='.'
data['dayofweek_median']='.'
import calendar
data['weekday']=data['timestamp'].apply(lambda x: calendar.weekday(x.year,x.month,x.day))

In [206]:
def func(total):
    if total['weekday']==0:
        return int(data.groupby(['weekday']).mean().iloc[0,0])
    elif total['weekday']==1: 
        return int(data.groupby(['weekday']).mean().iloc[1,0])
    elif total['weekday']==2: 
        return int(data.groupby(['weekday']).mean().iloc[2,0])
    elif total['weekday']==3: 
        return int(data.groupby(['weekday']).mean().iloc[3,0])
    elif total['weekday']==4: 
        return int(data.groupby(['weekday']).mean().iloc[4,0])
    elif total['weekday']==5: 
        return int(data.groupby(['weekday']).mean().iloc[5,0])
    elif total['weekday']==6: 
        return int(data.groupby(['weekday']).mean().iloc[6,0])
    else:
        return 0
        pass 
    
data.dayofweek_mean=data.apply(func,axis=1)

In [207]:
def func(total):
    if total['weekday']==0:
        return int(data.groupby(['weekday']).median().iloc[0,0])
    elif total['weekday']==1: 
        return int(data.groupby(['weekday']).median().iloc[1,0])
    elif total['weekday']==2: 
        return int(data.groupby(['weekday']).median().iloc[2,0])
    elif total['weekday']==3: 
        return int(data.groupby(['weekday']).median().iloc[3,0])
    elif total['weekday']==4: 
        return int(data.groupby(['weekday']).median().iloc[4,0])
    elif total['weekday']==5: 
        return int(data.groupby(['weekday']).median().iloc[5,0])
    elif total['weekday']==6: 
        return int(data.groupby(['weekday']).median().iloc[6,0])
    else:
        return 0
        pass 
    
data.dayofweek_median=data.apply(func,axis=1)

In [208]:
# 과거 temperature    
from bs4 import BeautifulSoup
import requests
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=279&yy={year}&obs=08&x=32&y=9'.format(year='2018'))  #천안 232 횡성 114 서울 108 김천(구미) 279
soup=BeautifulSoup(bam.text,'html.parser')
content=soup.find_all('td')
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=279&yy={year}&obs=08&x=32&y=9'.format(year='2019'))  
soup=BeautifulSoup(bam.text,'html.parser')
content1=soup.find_all('td')
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=279&yy={year}&obs=08&x=32&y=9'.format(year='2020'))  
soup=BeautifulSoup(bam.text,'html.parser')
content2=soup.find_all('td')
data['temperature']=data['timestamp'].apply(lambda x: content[3+(x.day-1)*13+x.month].text if x.year==2018 else (content1[3+(x.day-1)*13+x.month].text if x.year==2019 else content2[3+(x.day-1)*13+x.month].text ))


# 과거 강수량   
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=279&yy=2018&obs=21&x=18&y=7')  
soup=BeautifulSoup(bam.text,'html.parser')
content=soup.find_all('td')
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=279&yy=2019&obs=21&x=18&y=7')  
soup=BeautifulSoup(bam.text,'html.parser')
content1=soup.find_all('td')
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=279&yy=2020&obs=21&x=18&y=7')  
soup=BeautifulSoup(bam.text,'html.parser')
content2=soup.find_all('td')
data['rainfall']=data['timestamp'].apply(lambda x: content[3+(x.day-1)*13+x.month].text if x.year==2018 else (content1[3+(x.day-1)*13+x.month].text if x.year==2019 else content2[3+(x.day-1)*13+x.month].text ))
data['rainfall']=data['rainfall'].apply(lambda x: float(x.replace('\xa0','0')))

import numpy as np
def abnormal(prev):
    try: 
        if (prev['timestamp'].month in [6,7,8]) & (float(prev['temperature'])>=28):
            return 1
        elif (prev['timestamp'].month in [9,10,11]) & (float(prev['temperature'])>=5):
            return 1
        elif (prev['timestamp'].month in [12,1,2]) & (float(prev['temperature'])<=-5):
            return 1
        elif (prev['timestamp'].month in [3,4,5]) & (float(prev['temperature'])<=5):
            return 1
        else:
            return 0
    except:
        prev['temperature']==np.nan
            
data['abnormal']=data.apply(abnormal,axis=1) 


In [209]:
holiday=[]
date=['01','02','03','04','05','06','07','08','09','10','11','12']
for j in range(2019,2021):
    for i in range(len(date)):
        url='http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear={year}&solMonth={date}&ServiceKey=J%2FYE3sN528TkttbFXsmjXDfVrpZyKNjduV1r%2FcUxAfJ7wHnrjO2F5zzkJx2JvBHsmuNstAVe%2B5n6fS2dNgcKvQ%3D%3D'.format(year=str(j),date=date[i])
        resp=requests.get(url)
        soup=BeautifulSoup(resp.content)
        Text=soup.find_all('locdate')
        for i in range(len(Text)):
            holiday.append(pd.to_datetime(Text[i].text)) 
            
def holiday1(x):
    if (x in holiday)==False:
        return 0
    else:
        return 1
    
from datetime import timedelta    
def before_holiday(data):
    if data['holiday']==1:
        return 0
    elif data['timestamp']+timedelta(days=1) in holiday:
        return 1
    else:
        return 0    

def after_holiday(data):
    if data['holiday']==1:
        return 0
    elif data['timestamp']+timedelta(days=-1) in holiday:
        return 1
    else:
        return 0 
        
data['holiday']=data['timestamp'].apply(lambda x: holiday1(x))
data['before_holiday']=data.apply(before_holiday,axis=1)
data['after_holiday']=data.apply(after_holiday,axis=1)   

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [210]:
import openpyxl
import pandas as pd
import pandas as pd
present=pd.read_excel("본사_천안_횡성_김천_출근 내역(2018~2020).xlsx",index_col=0,sheet_name='김천')   

In [211]:
present=present.reset_index()
present=present[(present['WORK_DATE']>='2019-01-01')&(present['WORK_DATE']<='2020-11-30')]
present=present.rename(columns={'WORK_DATE':'timestamp','인원 수':'gate'})
present=present.reset_index().drop('index',axis=1)
present.drop('timestamp',axis=1)
present['timestamp']=present['timestamp'].apply(lambda x:pd.to_datetime(x))

In [212]:
data=data.reset_index().drop('index',axis=1) 
data=pd.merge(data,present,on='timestamp',how='left')
data=data.fillna(0)
data=data.reset_index().drop('index',axis=1)

In [213]:
data=data[['timestamp', 'target_value', 'dayofweek_mean', 'dayofweek_median', 'weekday', 'temperature', 'rainfall', 'holiday', 'before_holiday', 'after_holiday','abnormal','gate']]
data=data.fillna(0)

In [214]:
data.to_csv('김천input.csv',index=False)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
data=pd.read_csv('횡성 input.csv')
y=data[['중식']]
X=data[data.columns.difference(['timestamp','중식','석식','야식','조식'])]
X['rainfall']=X['rainfall'].apply(lambda x: float(x.replace('\xa0','0')))
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X=scaler.fit_transform(X)
y=scaler.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(603, 10)
(67, 10)
(603, 1)
(67, 1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [2]:
X_train=X_train.reshape((-1,1,10))
X_test=X_test.reshape((-1,1,10))
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(603, 1, 10)
(67, 1, 10)
(603, 1)
(67, 1)


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Input, Dense
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional, Activation
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

model = Sequential()
model.add(Bidirectional(LSTM(20, return_sequences = False), input_shape = (1,10)))
model.add(Dense(1))
model.add(Activation('softmax'))
adam = optimizers.Adam(lr = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) 

# model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 200, batch_size = 50, verbose = 2)
model.fit(X_train, y_train,epochs = 200, batch_size = 50, verbose = 2)
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)
print(accuracy_score(y_pred, y_test_)) 
RMSE = mean_squared_error(y_test, y_pred)**0.5
RMSE

Epoch 1/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 2/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 3/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 4/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 5/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 6/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 7/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 8/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 9/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 10/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 11/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 12/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 13/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 14/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 15/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 16/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 17/200
13/1

0.5672534841560553

In [ ]:
첨단의왕, 푸드본사, 김천, 횡성